전이학습 - vgg16

In [1]:
import torch
import torch.nn as nn
from torchvision.models.vgg import vgg16

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [27]:
# 전이학습 모델 로드
model = vgg16(weights=True).to(device)  # 기존 가중치를 사용하고 출력만 원하는 형태로 변경
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [28]:
# 가중치 고정
for param in model.parameters():
  param.requires_grad = False

In [29]:
fc = nn.Sequential(
      nn.Linear(in_features=25088, out_features=4096, bias=True),
      nn.ReLU(inplace=True),
      nn.Dropout(p=0.5, inplace=False),
      nn.Linear(in_features=4096, out_features=4096, bias=True),
      nn.ReLU(inplace=True),
      nn.Dropout(p=0.5, inplace=False),
      nn.Linear(in_features=4096, out_features=10, bias=True)
    )
model.classifier = fc
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [5]:
from torchvision.datasets.cifar import CIFAR10
from torchvision.transforms import Compose, ToTensor,Resize, RandomHorizontalFlip,RandomCrop, Normalize
from torch.utils.data import DataLoader

In [6]:
transforms = Compose([
    Resize((224,224)),
    RandomHorizontalFlip(),
    RandomCrop((224,224),padding=4),
    ToTensor(),
    Normalize((0.5,0.5,0.5),(0.25,0.25,0.25))  # RGB에서 각각에대한 평균관 표준편차를 이용해서 픽셀의 값을 정규화
])

데이터 로더 정의

In [41]:
training_data = CIFAR10(    root='./data',    train=True,    download=True,    transform=transforms)
test_data = CIFAR10(    root='./data',    train=False,    download=True,    transform=transforms)
train_loader = DataLoader(training_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
for img,label in training_data:
  print(img.shape)
  break

for img,label in train_loader:
  print(img.shape)
  break

torch.Size([3, 224, 224])
torch.Size([32, 3, 224, 224])


- 학습루프

In [9]:
# 학습률
# 최적화 옵티마이져 종류
# 에포크만큰 순환
  # 데이터로드 만큰 순환
    # 기울기 초기화
    # 기 정의된 모델을 통해 값을 예측
    # 손실함수를 정의한후 예측값과 정답을 통해서 오차를 계산 이때 손실함수를 통해서 구한 오차는 텐서객체
    # 오차는 backward() 가중치 업데이트 준비 계산방향을 전방계산에서 역전파로 변경
    # 최적화인 옵티마이져를 통해서 각 계산단계별 기울기값을 가지고 가중치를 업데이트
    # 필요하면 단계별로 오차를 출력해서 학습상태를 모니터링 한다
# 모델을 저장하고
# 모델을 불러와서 평가

In [30]:
# 모델을 사용하려면 사용하려는 device에 올려놓는다
model.to(device)
lr = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()  # 오차함수 정의
# 학습루프
from tqdm import tqdm

for epoch in range(1):
  iterator = tqdm(train_loader)
  for data, label in iterator:
    data = data.to(device)
    label = label.to(device)
    optimizer.zero_grad() # 기울기 초기화

    output = model(data) # 예측
    loss = loss_fn(output, label) # 오차계산
    loss.backward() # 기울기 계산
    optimizer.step() # 가중치 업데이트

    iterator.set_description(f"epoch:{epoch+1} loss:{loss.item()}")
# 모델 저장
torch.save(model.state_dict(),'/content/drive/MyDrive/model/CIFAR_pretrained.pth')

epoch:1 loss:0.9721008539199829: 100%|██████████| 1563/1563 [06:29<00:00,  4.01it/s]


In [31]:
# 모델 저장
torch.save(model.state_dict(),'/content/drive/MyDrive/model/CIFAR_pretrained.pth')

In [32]:
# 모델 불러오기
model.load_state_dict(torch.load('/content/drive/MyDrive/model/CIFAR_pretrained.pth'))
num_corr = 0
with torch.no_grad():
  for data, label in test_loader:
    data = data.to(device); label = label.to(device)
    output = model(data)
    # print(output.max(1)[1])  # 텐서의 max는 최대값과 인덱스위츠를 텐서형태로 반환 [1] 이 값이 인덱스위치
    # break
    preds = output.max(1)[1]
    corr = preds.eq(label)
    num_corr +=corr.sum()
  print(f'accuracy:{num_corr / len(test_data)}')

<ipython-input-32-e99bfef08d54>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/model/CIFAR_pretrained.pth'))


accuracy:0.7419999837875366


ResNet 모델 만들기

In [71]:
# ResNet 기본 블럭
import torch
import torch.nn as nn
class BasciBlock(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size=3):
    super(BasciBlock,self).__init__()
    # 합성곱 층 정의
    self.conv1 = nn.Conv2d(in_channels, out_channels,kernel_size=kernel_size,padding=1)
    self.conv2 = nn.Conv2d(out_channels, out_channels,kernel_size=kernel_size,padding=1)
    self.downsample = nn.Conv2d(in_channels,out_channels,kernel_size=1)
    # 배치정규화
    self.bn1 = nn.BatchNorm2d(out_channels)
    self.bn2 = nn.BatchNorm2d(out_channels)
    # 활성화 함수
    self.relu = nn.ReLU()
  def forward(self,x):
    x_ = x
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.conv2(x)
    x = self.bn2(x)
    x_ = self.downsample(x_)
    x += x_
    x = self.relu(x)
    return x

In [70]:
28*28*16

12544

In [73]:
class ResNet(nn.Module):
  def __init__(self, num_class = 10):
    super(ResNet,self).__init__()
    # 기본블럭
    self.b1 = BasciBlock(in_channels=3, out_channels=64)
    self.b2 = BasciBlock(in_channels=64, out_channels=128)
    self.b3 = BasciBlock(in_channels=128, out_channels=16)
    # 폴링
    self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
    # 분류기 FC
    self.fc1 = nn.Linear(in_features=28*28*16, out_features=2048)
    self.fc2 = nn.Linear(in_features=2048, out_features=512)
    self.fc3 = nn.Linear(in_features=512, out_features=num_class)
    # 활성화 함수
    self.relu = nn.ReLU()
  def forward(self,x):
    x = self.pool( self.b1(x) )
    x = self.pool( self.b2(x) )
    x = self.pool( self.b3(x) )
    # 분류기
    x = torch.flatten(x, start_dim=1)  # 시작차수  (배치사이즈, 채널,가로,세로)
    x = self.relu(self.fc1(x))
    x = self.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [74]:
# 모델 구현
model = ResNet(num_class=10).to(device)
model

ResNet(
  (b1): BasciBlock(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (downsample): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (b2): BasciBlock(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (downsample): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (b3): BasciBlock(
    (conv1): Conv2d(128, 16, kernel_size=(3, 3), stride=(1, 1), pa

In [ ]:
# # 에러 핸들링
# temp = BasciBlock(in_channels=3, out_channels=64)
# for data, label in train_loader:
#   temp(data)
#   break

In [76]:
# 학습
lr = 1e-4
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
for epoch in range(1):
  iterator = tqdm(train_loader)
  for data,label in iterator:
    data = data.to(device); label = label.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = loss_fn(output, label)
    loss.backward()
    optimizer.step()
    iterator.set_description(f"epoch:{epoch+1} loss:{loss.item()}")
torch.save(model.state_dict(), "/content/drive/MyDrive/model/ResNet.pth")

epoch:1 loss:1.8801732063293457: 100%|██████████| 1563/1563 [10:41<00:00,  2.44it/s]


In [77]:
# 모델 불러오기
model.load_state_dict(torch.load('/content/drive/MyDrive/model/ResNet.pth'))
num_corr = 0
with torch.no_grad():
  for data, label in test_loader:
    data = data.to(device); label = label.to(device)
    output = model(data)
    # print(output.max(1)[1])  # 텐서의 max는 최대값과 인덱스위츠를 텐서형태로 반환 [1] 이 값이 인덱스위치
    # break
    preds = output.max(1)[1]
    corr = preds.eq(label)
    num_corr +=corr.sum()
  print(f'accuracy:{num_corr / len(test_data)}')

<ipython-input-77-4949cb3dff1d>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/model/ResNet.pth'))


accuracy:0.5942999720573425


ResNet  전이학습

In [8]:
import torch
import torch.nn as nn
from torchvision.models.resnet import resnet50

In [3]:
# 전이학습 VGG16 이용해서 Resnet50 전이학습 파인튜닝으로

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
resnet = resnet50(weights=True).to(device)
resnet

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [11]:
# 파인튜닝을 위해서는 가중치를 고정
for param in resnet.parameters():
  param.requires_grad = False

In [12]:
# 모델의 분류기를 10개를 분류하는 FC로 변경
resnet.fc = nn.Linear(in_features=2048, out_features=10, bias=True)
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [13]:
# 데이터준비..
# transform을 이용해서 입력데이터의 크기를 고정시킨다.. 224
from torchvision.datasets.cifar import CIFAR10
from torchvision.transforms import Compose, ToTensor,Resize, RandomHorizontalFlip,RandomCrop, Normalize
from torch.utils.data import DataLoader
transforms = Compose([
    Resize((224,224)),
    RandomHorizontalFlip(),
    RandomCrop((224,224),padding=4),
    ToTensor(),
    Normalize((0.5,0.5,0.5),(0.25,0.25,0.25))  # RGB에서 각각에대한 평균관 표준편차를 이용해서 픽셀의 값을 정규화
])
training_data = CIFAR10(    root='./data',    train=True,    download=True,    transform=transforms)
test_data = CIFAR10(    root='./data',    train=False,    download=True,    transform=transforms)
train_loader = DataLoader(training_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

100%|██████████| 170498071/170498071 [00:04<00:00, 42463236.98it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
from tqdm import tqdm
# 학습
lr = 1e-4
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet.parameters(), lr=lr)
for epoch in range(1):
  iterator = tqdm(train_loader)
  for data,label in iterator:
    data = data.to(device); label = label.to(device)
    optimizer.zero_grad()
    output = resnet(data)
    loss = loss_fn(output, label)
    loss.backward()
    optimizer.step()
    iterator.set_description(f"epoch:{epoch+1} loss:{loss.item()}")
torch.save(resnet.state_dict(), "/content/drive/MyDrive/model/ResNet50.pth")

epoch:1 loss:2.1594016551971436:   1%|▏         | 21/1563 [03:24<4:26:06, 10.35s/it]

In [ ]:
# 평가 코드는 위에서 했던 코드를 재사용(이때 모델을 resnet으로  변경)